In [1]:
%%bash 
rsync -r LRMfwonLocalhost:/home/debian/test . 

ssh: connect to host localhost port 2211: Connection refused
rsync: connection unexpectedly closed (0 bytes received so far) [Receiver]
rsync error: error in rsync protocol data stream (code 12) at io.c(235) [Receiver=3.1.2]


In [2]:
import sys
sys.path.insert(0, "/home/esr/LRMessanlage/pyLRM")
from pathlib import Path 
import parse
import pandas as pd
import datetime
from bokeh.models import ColumnDataSource,DatetimeTickFormatter
from bokeh.plotting import figure, show, output_notebook
from logging_handler import ax_sensor_log_parser,_ax_sensor_log_row_parser

output_notebook()

def calc_pby(df,time_threshold_s):
    """
    diese funktion splittet eine df von achsdaten in mehrere unterlisten falls 
    die Zeit zwichen Achsdaten Ereignisse > ist als den angegebene time_threshold_s
    """
    pby = []
    pbycnt=0
    ax = []
    axcnt=0
    for toff in df.time_wheel_off.values:
        if toff> time_threshold_s:
            axcnt=0
            pbycnt+=1
        else:
            axcnt+=1
        pby.append(pbycnt)
        ax.append(axcnt)
    return df.assign(passby=pby).assign(ax=ax)

Loading BokehJS ...

### Messung Pfad und name  

In [3]:
path= Path("/home/esr/Documents/Luzern/")
mess_p=Path("/home/esr/Documents/Luzern/Nachmessung_23-11/")
log_p=mess_p.joinpath("LRM_messung.log")
rec_p = path.joinpath("KG")
print("{} exists: {}.".format(log_p.as_posix(),log_p.exists()))

/home/esr/Documents/Luzern/Nachmessung_23-11/LRM_messung.log exists: True.


### Lade achs data
die achs daten sind vom LRM_messung.log abgeholt.


In [4]:
ax_data = pd.DataFrame(ax_sensor_log_parser(log_p)['MSG_HEADER_AXLE']).T

### Filtern anahand daten intervalle

In [9]:
zeitintervalle = [
    (pd.Timestamp("2018-11-13 07:28:00"),pd.Timestamp("2018-11-30 22:00:00")),
                 ]
#between_times= ['8:00', '12:00']

In [10]:
ax_df = pd.concat([ax_data[ti_start:ti_stop] for (ti_start,ti_stop) in zeitintervalle])
#ax_df = ax_df.between_time(*between_times)
ax_df.index.name="timestamp"

### Plot

In [11]:
p = figure(x_axis_type="datetime", plot_width=1500, plot_height=750,tools=['xwheel_zoom','xpan'], active_scroll='xwheel_zoom',active_drag='xpan')
p.scatter('timestamp', 'ax_number',marker='circle',alpha=0.5,size=5, source=ColumnDataSource(ax_df), color="orange")


p.xaxis.formatter = DatetimeTickFormatter(seconds=["%H:%M:%S"],
                                            minutes=["%H:%M"],
                                            minsec=["%H:%M:%S"],
                                            hours=["%H"])
show(p)

### Plot2
falls die geloggte achzahlen nicht in passby unterteilt wurden kann mandie funktion `calc_pby` angewenden um passby zu unterscheiden

In [12]:
df_km94_2 =ax_df[ax_df.from_axle_sensor=="km94_2"].sort_index()
df_km94_4 =ax_df[ax_df.from_axle_sensor=="km94_4"].sort_index()
pb_km94_2=calc_pby(df_km94_2,10)
pb_km94_4=calc_pby(df_km94_4,10)

In [13]:
p = figure(x_axis_type="datetime", plot_width=1500, plot_height=750,tools=['xwheel_zoom','xpan'], active_scroll='xwheel_zoom',active_drag='xpan')
p.scatter('timestamp', 'ax',marker='circle',alpha=0.5,size=5, source=ColumnDataSource(pb_km94_2),legend="km94_2(906D)", color="orange")
p.scatter('timestamp', 'ax',marker="square",alpha=0.5,size=5,source=ColumnDataSource(pb_km94_4),legend="km94_4(3136)(SSA)", color="green")


p.xaxis.formatter = DatetimeTickFormatter(seconds=["%H:%M:%S"],
                                            minutes=["%H:%M"],
                                            minsec=["%H:%M:%S"],
                                            hours=["%H"])
show(p)